In [6]:
import pyarrow as pa

# Unregister if already exists (avoids ArrowKeyError)
try:
    pa.unregister_extension_type("pandas.period")
except (KeyError, pa.ArrowKeyError):
    pass

try:
    pa.unregister_extension_type("pandas.interval")
except (KeyError, pa.ArrowKeyError):
    pass

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
data = pd.read_parquet('../data/met_forecast.parquet',engine="fastparquet")

In [13]:
data.head()


,sid,time_ref,time,lt,ws10m_00,ws10m_01,ws10m_02,ws10m_03,ws10m_04,ws10m_05,...,g10m_05,g10m_06,g10m_07,g10m_08,g10m_09,g10m_10,g10m_11,g10m_12,g10m_13,g10m_14
0,Engerfjellet,2020-02-15 12:00:00,2020-02-15 12:00:00,0,2.096957,2.110652,2.304629,1.942044,1.447011,2.288181,...,6.918935,8.110546,5.988888,6.934274,5.235212,5.507061,7.193576,6.753938,4.841811,4.875890
1,Engerfjellet,2020-02-15 12:00:00,2020-02-15 13:00:00,1,3.449402,3.436193,3.205872,3.450461,3.143307,4.278822,...,9.914339,9.184751,6.999992,7.892150,7.214608,8.252510,9.535547,8.717684,8.152432,6.240438
2,Engerfjellet,2020-02-15 12:00:00,2020-02-15 14:00:00,2,3.348926,2.903557,2.779971,3.733802,3.148597,2.912592,...,10.031385,8.624619,7.984353,8.264300,7.205638,9.052546,9.531441,8.367238,7.840395,7.153080
3,Engerfjellet,2020-02-15 12:00:00,2020-02-15 15:00:00,3,3.095971,2.560161,2.448126,2.267286,3.050008,3.310141,...,8.185250,8.821701,7.481009,8.172009,6.255360,7.345034,8.303812,6.735660,7.600200,7.189164
4,Engerfjellet,2020-02-15 12:00:00,2020-02-15 16:00:00,4,2.821545,2.965743,3.235484,3.212730,3.064931,2.420390,...,7.551644,9.709259,8.365305,8.703517,6.151643,10.116440,7.346225,9.079601,9.048587,6.937930


In [14]:
data_nowcast = pd.read_parquet('../data/met_nowcast.parquet',engine="fastparquet")
data_nowcast.head()

,air_temperature_2m,air_pressure_at_sea_level,relative_humidity_2m,precipitation_amount,wind_speed_10m,wind_direction_10m,windpark
time,,,,,,,
2020-01-01 00:00:00,276.363831,100703.859375,0.966131,3.147963,13.445930,250.681976,Bessakerfjellet
2020-01-01 01:00:00,276.951080,100595.609375,0.972677,3.636308,13.807796,251.660736,Bessakerfjellet
2020-01-01 02:00:00,277.113007,100526.656250,0.964682,1.921109,15.788631,250.626617,Bessakerfjellet
2020-01-01 03:00:00,277.259033,100408.359375,0.966498,1.919362,16.493980,254.728180,Bessakerfjellet
2020-01-01 04:00:00,277.229248,100421.062500,0.964835,1.275917,16.933884,253.571869,Bessakerfjellet


In [ ]:
import pyarrow.parquet as pq

table = pq.read_table("../data/wind_power_per_bidzone.parquet")
data_bidzone = table.to_pandas(ignore_metadata=True)   # <- key bit
# If you know the intended index column, set it explicitly:
# data_bidzone = data_bidzone.set_index("timestamp")  # example
data_bidzone.head()

,ELSPOT NO1,ELSPOT NO2,ELSPOT NO3,ELSPOT NO4,__index_level_0__
0,149.285262,434.753864,497.143482,152.830002,2020-01-01 00:00:00
1,152.634024,395.572485,471.916899,144.440344,2020-01-01 01:00:00
2,151.163256,355.408523,399.325698,143.960894,2020-01-01 02:00:00
3,150.223341,375.316902,367.662849,134.310523,2020-01-01 03:00:00
4,157.415142,412.949107,399.627337,88.573315,2020-01-01 04:00:00


In [23]:
data_windparks = pd.read_csv('../data/windparks_bidzone.csv')
data_windparks.head()

,bidding_area,substation_name,operating_power_max,prod_start_new,eic_code
0,ELSPOT NO1,Engerfjellet,52.8,2022-09-19 13:00:00,50WP00000002158R
1,ELSPOT NO1,Hån Vindpark,21.0,2022-10-26 22:00:00,50WP000000022251
2,ELSPOT NO1,Kjølberget,55.9,2020-07-07 09:00:00,50WP00000002085S
3,ELSPOT NO1,Marker Vindpark,54.0,2020-01-01 00:00:00,50WP00000002048Y
4,ELSPOT NO1,Raskiftet,111.6,2020-01-01 00:00:00,50WP00000001718K
